28k imagens + Dropout

In [ ]:
class Model: 
    instance: Sequential 
    def __init__(self, s: Sequential): 
        self.instance = models.Sequential()

    def configure(self, img_size:int = 192, kernel_size = 3, color_channel = 3) -> None:
        
        self.instance.add(layers.Conv2D(128, kernel_size=(kernel_size, kernel_size), activation='relu', input_shape=(img_size, img_size, color_channel))) # 3 é sobre RGB
        #self.instance.add(layers.BatchNormalization())
        self.instance.add(layers.MaxPooling2D((2, 2)))
        
        self.instance.add(layers.Conv2D(64, kernel_size=(kernel_size, kernel_size), activation='relu'))
        #self.instance.add(layers.BatchNormalization())
        self.instance.add(layers.MaxPooling2D((2, 2)))
        
        self.instance.add(layers.Conv2D(32, kernel_size=(kernel_size, kernel_size), activation='relu'))
        #self.instance.add(layers.BatchNormalization())
        self.instance.add(layers.Dropout(0.5))
        
        self.instance.add(layers.Flatten())
        self.instance.add(layers.Dropout(0.5))
        
        self.instance.add(layers.Dense(16, activation='relu'))
        #self.instance.add(layers.BatchNormalization())
        self.instance.add(layers.Dropout(0.5))
        
        self.instance.add(layers.Dense(2))#activation="softmax"))
        self.instance.summary()
    
    def compile(self, optimizer:str = "adam") -> None:
        self.instance.compile(
            optimizer=optimizer,
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )
        
    def Run(self, rd: ResultData,epochs = 30, batch_size:int = 10):
        #datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        #    rotation_range=10,
        #    width_shift_range=0.05,
        #    height_shift_range=0.05,
        #    brightness_range=(0.8, 1.2),
        #    horizontal_flip=True
        #)
        callback = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
        history =  self.instance.fit( 
            rd.train_images, 
            rd.train_labels, 
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(rd.test_images, rd.test_labels),
            #callback=[callback]
        )
        return history
    
    
    def Testit(self, rd: ResultData) -> float:
        print("[INFO]: Generating test predictions...")
       
        # make class predictions
        #predictions = (self.instance.predict(test_images) > 0.5).astype(int)
        predictions = self.instance.predict(rd.test_images)
        # returns index of the maximum value in prediction array [0.0,0.0,1.0] -> 2
        y_pred_class = np.argmax(predictions, axis=1)
        y_test_class = np.argmax(rd.test_labels, axis=1)
    
        l = len(y_test_class)
        acc = sum([y_pred_class[i]==y_test_class[i] for i in range(l)])/l
        print('Accuracy: %.2f %%' % (acc*100))
    
        f1score = 0
        f1score += f1_score(y_test_class,y_pred_class,average="weighted")
        print("F_measure: ",round(100*f1score, 2),"%")

        print("Classes verdadeiras únicas:", np.unique(y_test_class))
        print("Classes preditas únicas:", np.unique(y_pred_class))
        #Confusion Matrix
        cm = confusion_matrix(y_test_class, y_pred_class)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.show()
        return acc
        
    def Save(self,accuracy:float):
        if (accuracy > 0.5):
            self.instance.save(r'./kaggle/working/dice.keras')

In [ ]:
def main():
    
    base_path = "/kaggle/input/solarpanelpowerloss/data"
    num_images_per_class:int = 14000
    assert(num_images_per_class < 14500)
    assert(num_images_per_class % 4 == 0)

    paths:list[str] = Load20kPaths(base_path,num_images_per_class)
    # 0   1
    # 1   2
    
    m: Model = Model(Sequential)
    m.configure()
    m.compile()
    accuracy:float = 0.0
    
    for i in range(0,4):
        rd = LoadAndSplitDataset( 
                paths[int((i/2)*num_images_per_class):int( ((1+i)/2)*num_images_per_class)],
                base_path,
                num_images_per_class
            ) 
        ShowPics(rd.train_images, rd.train_labels)
        history = m.Run(rd, 10)
        PlotTrainingAccuracy(history)
        accuracy = m.Testit(rd)
        
        
    m.Save(accuracy)
    

main()